This is a live notebook with experimental code to develop functionality for loading E4 data.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy as sp
import scipy.stats
import pandas as pd
import numpy as np
from tqdm import tqdm
import itertools
import re
res_digit = r'[0-9]'

import mne
import json



In [ ]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")




In [ ]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import STUDY_DATA_FOLDER
from predicament.utils.config import E4_LOCAL_DIRPATHS
from predicament.utils.config import E4_CSV_FILES

from predicament.data.timeseries import load_E4_csv_data
from predicament.data.timeseries import merge_E4_csv_data
from predicament.data.timeseries import create_participant_data_E4_only
from predicament.data.timeseries import create_participant_data

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition


In [ ]:

E4_full_dirpaths = {
    participant: os.path.join(STUDY_DATA_FOLDER,local_path)
        for participant, local_path in E4_LOCAL_DIRPATHS.items() }
E4_full_dirpaths

In [ ]:
E4_CSV_FILES

In [ ]:
full_dirpath = E4_full_dirpaths['VG_01']

csv_data = load_E4_csv_data(full_dirpath, csv_files=E4_CSV_FILES)
for k, v in csv_data.items():
    print(f"csv_fname = {k}")
    print(f"start_time = {v['start_time']}")
    print(f"sample_rate = {v['sample_rate']}")
    print(f"timeseries.shape = {v['timeseries'].shape}")
    print(f"timeseries = {v['timeseries']}")
    print()

In [ ]:

data, channel_names, sample_rate, std_start_time = merge_E4_csv_data(csv_data, csv_files=E4_CSV_FILES)
print(f"std_start_time = {std_start_time}")
print(f"channel_names = {channel_names}")
print(f"sample_rate = {sample_rate}")
print(f"data.shape = {data.shape}")
print(f"data = {data}")

In [ ]:
all_participants_data = create_participant_data_E4_only()


## Data description

From the file '/CARE_HOME_DATA/VG01/E4_8921_15_44/info.txt':

.csv files in this archive are in the following format:
The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.

TEMP.csv
Data from temperature sensor expressed degrees on the Celsius (°C) scale.

EDA.csv
Data from the electrodermal activity sensor expressed as microsiemens (μS).

BVP.csv
Data from photoplethysmograph.

ACC.csv
Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore the unit in this file is 1/64g.
Data from x, y, and z axis are respectively in first, second, and third column.

IBI.csv
Time between individuals heart beats extracted from the BVP signal.
No sample rate is needed for this file.
The first column is the time (respect to the initial time) of the detected inter-beat interval expressed in seconds (s).
The second column is the duration in seconds (s) of the detected inter-beat interval (i.e., the distance in seconds from the previous beat).

HR.csv
Average heart rate extracted from the BVP signal.The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.


tags.csv
Event mark times.
Each row corresponds to a physical button press on the device; the same time as the status LED is first illuminated.
The time is expressed as a unix timestamp in UTC and it is synchronized with initial time of the session indicated in the related data files from the corresponding session.

In [ ]:
all_participants_data['VG_01'].event_details.events_info

In [ ]:
all_participants_data['VG_01'].event_details.exp_start_time